In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab 
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx
import os
from sklearn.model_selection import train_test_split
import load_and_match as lam

2024-11-12 17:52:44.706968: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 17:52:44.784266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/04


In [2]:
# Set matplotlib default color cycle
new_color_cycle = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#aec7e8',
    '#ffbb78',
    '#98df8a',
    '#ff9896',
    '#c5b0d5',
    '#c49c94',
    '#f7b6d2',
    '#c7c7c7',
    '#dbdb8d',
    '#9edae5'
]

# You can then apply this new color cycle to your matplotlib plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_color_cycle)

In [3]:
L1AD_rate = 1000
target_rate = 10

### Main

In [4]:
# Load and match my data with the topo2A (L1AD) scores. Only needs to be done once ever.
#lam.load_and_match('./h5_ntuples/11-5-2024')

In [ ]:
training_info = {
    "save_path": "../../../trained_models/trial_102", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    #"train_data_scheme": "topo2A_train", 
    "pt_normalization_type": "global_division", 
    "L1AD_rate": 1000,
    "comments": "train over run 475321"
}
datasets, data_info = ef.load_and_preprocess(**data_info)

In [ ]:
training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)

In [ ]:
ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

### old main

In [7]:
# data_info = {
#     #"train_data_scheme": "topo2A_train", 
#     "pt_normalization_type": "global_division", 
#     "L1AD_rate": 1000,
#     "comments": "train over combination of all three EB runs"
# }

# training_info = {
#     "save_path": "../../../trained_models/trial_101", 
#     "dropout_p": 0.1, 
#     "L2_reg_coupling": 0.01, 
#     "latent_dim": 4, 
#     "large_network": True, 
#     "num_trainings": 10,
#     "training_weights": True
# }

# datasets, data_info = ef.load_and_preprocess(**data_info)

Loaded A14N23LO from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/A14N23LO.h5
Loaded EB_473255 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_482596.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from /eos/home-m/mmcohen/git_repo

In [6]:
# training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)

Booting up... initializing trainings of 10 models

starting training model 0...

Epoch 20: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 29: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 35: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 11: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 31: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 38: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 44: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.

Epoch 50: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
model 0 success

starting training model 1...

Epoch 16: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 19: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 32: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch

2024-11-11 07:31:32.860326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-11 07:31:32.873695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-11 07:31:32.874001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
training_info = {
    "save_path": "../../../trained_models/trial_102", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    #"train_data_scheme": "topo2A_train", 
    "pt_normalization_type": "global_division", 
    "L1AD_rate": 1000,
    "comments": "train over run "
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'/plots_11-12-2024',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/A14N23LO.h5
Loaded EB_473255 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_482596.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HLT_noalg_eb_L1All.h5
Loaded ZZ

NameError: name 'sub_dict' is not defined

2024-11-12 16:29:32.166936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 16:29:32.183118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 16:29:32.183323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
# # Remake efficiency gain plots
# data_info = {
#     #"train_data_scheme": "topo2A_train", 
#     "pt_normalization_type": "global_division", 
#     "L1AD_rate": 1000,
#     "comments": "train over combination of all three EB runs"
# }

# training_info = {
#     "save_path": "../../../trained_models/trial_101", 
#     "dropout_p": 0.1, 
#     "L2_reg_coupling": 0.01, 
#     "latent_dim": 4, 
#     "large_network": True, 
#     "num_trainings": 10,
#     "training_weights": True
# }

# plots_path = training_info['save_path']+'/plots'
# ef.remake_distribution_plots(results_path=plots_path+'/stability_EG_results.json', plots_path=plots_path)

In [6]:
# plots_path = '../../../trained_models/trial_100/plots3'
# ef.remake_distribution_plots(results_path=plots_path+'/stability_EG_results.json', plots_path=plots_path)

### Test ONNX

In [7]:
training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_2", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    "train_data_scheme": 
    "topo2A_train+overlap", 
    "pt_normalization_type": 
    "global_division", 
    "L1AD_rate": 1000
}
model_version=0
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='HLT', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='L1', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
datasets, data_info = ef.load_and_preprocess(**data_info)
datasets = ef.compare_tf_with_onnx(datasets=datasets, training_info=training_info, model_version=0, onnx_path='./trained_models/multiple_trainings/trial_2/onnx')

Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:26.715362: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:26.715559: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:27.230193: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:27.230453: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX HLT_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/HLT_AE_0.onnx


Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:29.812788: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:29.812992: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:30.345443: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:30.345661: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX L1_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/L1_AE_0.onnx
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5


In [8]:
for tag, data_dict in datasets.items():
    print(f'{tag}:')
    for key, value in data_dict.items():
        print(f'    {key}: {value.shape}')

A14N23LO:
    HLT_data: (10000, 48)
    L1_data: (10000, 48)
    passHLT: (10000,)
    passL1: (10000,)
    topo2A_AD_scores: (10000,)
    weights: (10000,)
    L1Seeded: (10000,)
    HLT_model_outputs: (10000, 48)
    L1_model_outputs: (10000, 48)
    HLT_AD_scores: (10000,)
    L1_AD_scores: (10000,)
    ONNX_HLT_model_outputs: (10000, 48)
    ONNX_L1_model_outputs: (10000, 48)
    ONNX_HLT_AD_scores: (10000,)
    ONNX_L1_AD_scores: (10000,)
HAHMggfZdZd2l2nu:
    HLT_data: (70000, 48)
    L1_data: (70000, 48)
    passHLT: (70000,)
    passL1: (70000,)
    topo2A_AD_scores: (70000,)
    weights: (70000,)
    L1Seeded: (70000,)
    HLT_model_outputs: (70000, 48)
    L1_model_outputs: (70000, 48)
    HLT_AD_scores: (70000,)
    L1_AD_scores: (70000,)
    ONNX_HLT_model_outputs: (70000, 48)
    ONNX_L1_model_outputs: (70000, 48)
    ONNX_HLT_AD_scores: (70000,)
    ONNX_L1_AD_scores: (70000,)
HHbbttHadHad:
    HLT_data: (100000, 48)
    L1_data: (100000, 48)
    passHLT: (100000,)
    pa

In [9]:
print(datasets['EB_test']['HLT_AD_scores'][0:5])
print()
print(datasets['EB_test']['ONNX_HLT_AD_scores'][0:5])

[1.62544931 1.92602879 1.33561259 0.         1.06108748]

[1.62544931 1.92602879 1.33561259 0.         1.06108748]


In [ ]:
# Success! These AD scores are the same.

### Random tests

In [9]:
def load_subdicts_from_h5(save_dir):
    """
    Loads sub-dictionaries of NumPy arrays from HDF5 files in a directory and reconstructs the original structure.
    
    Args:
        save_dir (str): The directory where the HDF5 files are stored.
    
    Returns:
        main_dict (dict): A dictionary of dictionaries where the innermost values are NumPy arrays.
    """
    main_dict = {}
    
    for filename in os.listdir(save_dir):
        if filename.endswith(".h5"):
            sub_dict_name = os.path.splitext(filename)[0]
            file_path = os.path.join(save_dir, filename)
            with h5py.File(file_path, 'r') as f:
                sub_dict = {key: np.array(f[key]) for key in f}
            main_dict[sub_dict_name] = sub_dict
            print(f"Loaded {sub_dict_name} from {file_path}")
    
    return main_dict

In [15]:
datasets = load_subdicts_from_h5('/eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024')

for tag, data_dict in datasets.items():
        print(f'{tag}:')
        for key, value in data_dict.items():
            print(f'    {key}: {value.shape}')

Loaded A14N23LO from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/A14N23LO.h5
Loaded EB_473255 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/EB_482596.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/11-5-2024/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from /eos/home-m/mmcohen/git_repo

In [17]:
print(np.sum(datasets['EB_475321']['run_numbers']==475321) + np.sum(datasets['topo2A_train']['run_numbers']==475321))
print(np.sum(datasets['EB_475321']['run_numbers']==475321))

1119657
26772


In [11]:
datasets = load_subdicts_from_h5('/eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples')

for tag, data_dict in datasets.items():
        print(f'{tag}:')
        for key, value in data_dict.items():
            print(f'    {key}: {value.shape}')

Loaded A14N23LO from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/A14N23LO.h5
Loaded EB from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/EB.h5
Loaded EB_test2 from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/EB_test2.h5
Loaded EB_train from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from /eos/home-m/mmcohen/git_repos/AD_trigger_training/training_notebooks/cohen/h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs fr

In [14]:
print(np.sum(datasets['EB_test2']['run_numbers']==475321) + np.sum(datasets['EB_train']['run_numbers']==475321))

1092885


Okay so basically, run 475321 has so few events because most of them are in topo2A_train. So I have really low stats calculating rate, so the rate can't really be trusted in trial_100.